In [ ]:
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_08.csv')
date_columns = ['日期', '上市日期', '公告日期', '计划披露日', '计划开始日', '计划结束日', '离任日期']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

# Law Article 4

## MEU_4_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 计划通过本所集中竞价或大宗交易减持股份 |
| constrain | 应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划 |
| contextual_info | nan |
| note | 不考虑是否及时通知公司 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1725 |
| completion_tokens | 7141 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_4_1(df):
    '''
    检查MEU_4_1的合规性：
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    额外要求：不考虑是否及时通知公司
    '''

    df = df.copy()

    # 初始化标记列
    df['meu_4_1_subject'] = False
    df['meu_4_1_condition'] = False
    df['meu_4_1_constraint'] = False

    # 1. 标记valid的subject（上市公司大股东 | 董监高）
    # 大股东判断：控股股东/实际控制人/持股5%以上股东 或 持股比例>=5%
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    # 董监高判断：股东身份为董监高（不含离任）
    is_director = (df['股东身份'] == '董监高')
    valid_subject = is_major_shareholder | is_director

    # 2. 标记valid的condition（存在减持计划且减持方式为竞价/大宗）
    valid_condition = (
        df['存在减持计划'] & 
        df['减持方式'].isin(['竞价交易', '大宗交易'])
    )

    # 3. 标记valid的constraint（披露日早于开始日至少15自然日）
    # 计算自然日差（假设日期列已对齐交易日）
    days_diff = (df['计划开始日'] - df['计划披露日']).dt.days
    # 约束条件：存在减持计划且时间差>=15天
    valid_constraint = (
        df['存在减持计划'] & 
        (days_diff >= 15)
    )

    # 更新标记列
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True

    return df

In [ ]:
df = check_meu_4_1(df)
df

---

## MEU_4_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 披露减持计划 |
| constrain | 每次披露的减持计划中减持时间区间不得超过3个月 |
| contextual_info | nan |
| note | 三个月按照90个自然日计算 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1709 |
| completion_tokens | 5009 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_4_7(df):
    '''
    验证MEU_4_7合规性：
    - subject: 上市公司大股东或董监高
    - condition: 披露减持计划
    - constraint: 减持时间区间不超过90自然日
    '''
    df = df.copy()

    # 初始化合规性标记列
    df['meu_4_7_subject'] = False
    df['meu_4_7_condition'] = False
    df['meu_4_7_constraint'] = None

    # 1. 验证责任主体
    # 判断大股东（含持股5%以上）或董监高
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'] == '董监高'
    valid_subject = is_major_shareholder | is_director

    # 2. 验证触发条件
    valid_condition = df['存在减持计划']

    # 3. 验证时间区间约束（独立检查）
    # 计算自然日差并判断有效性
    has_valid_dates = df['计划开始日'].notna() & df['计划结束日'].notna()
    end_after_start = df['计划结束日'] >= df['计划开始日']
    days_diff = (df['计划结束日'] - df['计划开始日']).dt.days
    valid_constraint = has_valid_dates & end_after_start & (days_diff <= 90)

    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_7_subject'] = True
    df.loc[valid_condition, 'meu_4_7_condition'] = True
    df.loc[valid_constraint, 'meu_4_7_constraint'] = True
    df.loc[~valid_constraint, 'meu_4_7_constraint'] = False

    return df

In [ ]:
df = check_meu_4_7(df)
df

---

## MEU_4_8


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 拟在3个月内通过集中竞价交易减持股份的总数超过公司股份总数1% |
| constrain | 应当在首次卖出的30个交易日前预先披露减持计划 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1717 |
| completion_tokens | 7786 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_4_8(df):
    '''
    检查MEU_4_8合规性：
    subject: 上市公司大股东 | 董监高
    condition: 拟在3个月内通过集中竞价交易减持股份总数超1%
    constraint: 需在首次卖出30个交易日前披露计划
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_4_8_subject'] = False
    df['meu_4_8_condition'] = False
    df['meu_4_8_constraint'] = None  # 初始化为None以便区分未检查状态

    # 1. 验证责任主体 (subject)
    # 大股东标准：控股股东/实际控制人/持股5%以上股东或持股比例≥5%
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    # 董监高标准：直接身份标识
    is_director = df['股东身份'] == '董监高'
    valid_subject = is_major_shareholder | is_director
    df.loc[valid_subject, 'meu_4_8_subject'] = True

    # 2. 验证触发条件 (condition)
    # 集中竞价交易、存在减持计划、减持比例>1%、时间跨度≤90天
    valid_condition = (
        (df['减持方式'] == '竞价交易') &
        df['存在减持计划'] &
        (df['计划减持比例'] > 0.01) &
        ((df['计划结束日'] - df['计划开始日']).dt.days <= 90)
    )
    # 处理空值并标记
    valid_condition = valid_condition.fillna(False)
    df.loc[valid_condition, 'meu_4_8_condition'] = True

    # 3. 验证约束条件 (constraint)
    # 计算计划披露日与计划开始日的时间差
    days_diff = (df['计划开始日'] - df['计划披露日']).dt.days
    valid_constraint = days_diff >= 30
    # 空值处理并更新标记列
    valid_constraint = valid_constraint.fillna(False)
    df.loc[valid_constraint, 'meu_4_8_constraint'] = True
    df.loc[~valid_constraint, 'meu_4_8_constraint'] = False

    return df

In [ ]:
df = check_meu_4_8(df)
df

---

# Law Article 9

## MEU_9_1


| 字段 | 内容 |
|------|------|
| subject | 控股股东 | 实际控制人 | 董监高 |
| condition | 公司上市时未盈利且处于实现盈利前阶段 |
| constrain | 自公司股票上市之日起2个完整会计年度内不得减持公开发行并上市前股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1721 |
| completion_tokens | 7556 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_9_1(df):
    '''
    检查MEU_9_1合规性：
    subject: 控股股东 | 实际控制人 | 董监高
    condition: 公司上市时未盈利且处于实现盈利前阶段
    constraint: 自上市之日起2个完整会计年度内不得减持公开发行前股份
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_9_1_subject'] = False
    df['meu_9_1_condition'] = False
    df['meu_9_1_constraint'] = True  # 默认符合约束

    # 标记subject(独立检查)
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人', '董监高'])
    df.loc[valid_subject, 'meu_9_1_subject'] = True

    # 计算公司上市时净利润(按公司映射)
    company_ipo_profit = df.groupby('公司简称').apply(
        lambda x: x.set_index('日期').loc[x['上市日期'].iloc[0], '净利润']
    ).to_dict()
    df['_temp_ipo_profit'] = df['公司简称'].map(company_ipo_profit)

    # 计算两年后日期(自然年)
    df['_temp_two_year'] = df['上市日期'] + pd.DateOffset(years=2)

    # 标记condition(独立检查)
    ipo_unprofit = (df['_temp_ipo_profit'] <= 0)
    in_period = (df['日期'] <= df['_temp_two_year'])
    df['meu_9_1_condition'] = ipo_unprofit & in_period

    # 标记constraint(独立检查)
    has_sold = (df['当日减持比例'] > 0)
    df['meu_9_1_constraint'] = ~(in_period & has_sold)

    # 清理临时列
    df.drop(columns=['_temp_ipo_profit', '_temp_two_year'], inplace=True)
    
    return df

In [ ]:
df = check_meu_9_1(df)
df

---

## MEU_9_3


| 字段 | 内容 |
|------|------|
| subject | 董监高 |
| condition | 在前款规定的2个完整会计年度期间内离职 |
| constrain | 应当继续遵守前款规定的减持限制 |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_9_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1807 |
| completion_tokens | 9305 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_9_3(df):
    '''
    验证MEU_9_3合规性：
    subject: 董监高
    condition: 在MEU_9_1规定的2个完整会计年度期间内离职
    constraint: 继续遵守减持限制
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_9_3_subject'] = False
    df['meu_9_3_condition'] = False
    df['meu_9_3_constraint'] = None

    # 标记valid的subject（董监高或离任董监高）
    valid_subject = df['股东身份'].isin(['董监高', '离任董监高'])
    df.loc[valid_subject, 'meu_9_3_subject'] = True

    # 预处理公司基础信息
    company_info = df.groupby('公司简称').apply(
        lambda x: pd.Series({
            '上市日期': x['上市日期'].iloc[0],
            '上市时每股净利润': x.loc[x['日期'] == x['上市日期'].iloc[0], '每股净利润'].values[0] 
            if not x.loc[x['日期'] == x['上市日期'].iloc[0]].empty else None
        })
    ).reset_index()

    # 预处理股东离职信息
    resign_info = df.groupby(['公司简称', '股东']).apply(
        lambda x: pd.Series({
            '离任日期': x['离任日期'].dropna().iloc[0] 
            if not x['离任日期'].dropna().empty else None
        })
    ).reset_index()

    # 合并公司信息并计算时间窗口
    resign_info = resign_info.merge(company_info, on='公司简称', how='left')
    resign_info['两年结束日期'] = resign_info['上市日期'] + pd.DateOffset(years=2)
    
    # 获取离任时财务数据
    eps_ref = df[['公司简称', '日期', '每股净利润']].drop_duplicates()
    resign_info = resign_info.merge(
        eps_ref,
        left_on=['公司简称', '离任日期'],
        right_on=['公司简称', '日期'],
        how='left'
    ).rename(columns={'每股净利润': '离任时每股净利润'})

    # 计算condition有效性
    resign_info['condition_met'] = (
        (resign_info['上市时每股净利润'] < 0) &          # 上市时未盈利
        (resign_info['离任时每股净利润'] < 0) &        # 离职时未实现盈利
        (resign_info['离任日期'] <= resign_info['两年结束日期']) &  # 离职在时间窗口内
        (resign_info['离任日期'] >= resign_info['上市日期'])
    )

    # 合并condition标记到主表
    df = df.merge(
        resign_info[['公司简称', '股东', 'condition_met']],
        on=['公司简称', '股东'],
        how='left'
    )
    df['meu_9_3_condition'] = df['condition_met'].fillna(False)

    # 标记constraint有效性（所有行独立检查）
    df['上市日期'] = df.groupby('公司简称')['上市日期'].transform('first')
    df['两年结束日期'] = df['上市日期'] + pd.DateOffset(years=2)
    in_window = (df['日期'] >= df['上市日期']) & (df['日期'] <= df['两年结束日期'])
    has_reduce = df['当日减持比例'] > 0
    df['meu_9_3_constraint'] = ~(in_window & has_reduce)

    # 清理临时列
    df.drop(columns=['condition_met', '上市日期', '两年结束日期'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_meu_9_3(df)
df

---

# Law Article 10

## MEU_10_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉嫌与本上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1711 |
| completion_tokens | 3253 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_10_1(df):
    '''
    验证MEU_10_1合规性：
    "subject": "上市公司大股东",
    "condition": "因涉嫌与本上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间",
    "constraint": "不得减持其所持有的本公司股份"
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_10_1_subject'] = False
    df['meu_10_1_condition'] = False
    df['meu_10_1_constraint'] = None  # 初始化为None便于后续布尔类型填充

    # 1. 标记责任主体有效性(上市公司大股东)
    # 判断标准：股东身份属于大股东类别或持股比例≥5%
    major_shareholder_mask = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[major_shareholder_mask, 'meu_10_1_subject'] = True

    # 2. 标记触发条件有效性(处于被调查/侦查期间)
    # 判断标准：股东涉嫌事件包含立案调查/侦查状态
    investigation_status = [
        '被中国证监会及其派出机构立案调查',
        '被司法机关立案侦查'
    ]
    df['meu_10_1_condition'] = df['股东涉嫌证券期货违法犯罪事件'].isin(investigation_status)

    # 3. 标记约束有效性(未减持股份)
    # 判断标准：当日减持比例≤0（独立检查，不考虑主体和条件状态）
    df['meu_10_1_constraint'] = df['当日减持比例'] <= 0
    # 将布尔值转换为True/False显式标记（覆盖None初始值）
    df['meu_10_1_constraint'] = df['meu_10_1_constraint'].astype(bool)

    return df

In [ ]:
df = check_meu_10_1(df)
df

---

## MEU_10_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉嫌与本上市公司有关的证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1722 |
| completion_tokens | 8204 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_10_2(df):
    '''
    验证MEU_10_2合规性：
    - subject: 上市公司大股东（控股股东/实际控制人/持股5%以上股东）
    - condition: 因证券违法被调查且在处罚后6个月内
    - constraint: 不得减持股份
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_10_2_subject'] = False
    df['meu_10_2_condition'] = False
    df['meu_10_2_constraint'] = False

    # 1. 验证责任主体（上市公司大股东）
    subject_mask = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) | 
        (df['持股比例'] >= 0.05)
    )
    df.loc[subject_mask, 'meu_10_2_subject'] = True

    # 2. 验证触发条件（分组处理时间序列数据）
    # 预处理调查状态
    investigated_mask = df['股东涉嫌证券期货违法犯罪事件'].isin([
        '被中国证监会及其派出机构立案调查',
        '被司法机关立案侦查'
    ])
    has_investigated = df[investigated_mask].groupby(['公司简称', '股东']).size().reset_index()
    has_investigated['has_investigated'] = True

    # 预处理处罚时间点
    penalty_dates = df[df['股东涉嫌证券期货违法犯罪事件'].isin([
        '行政处罚决定作出',
        '司法机关立案侦查结束'
    ])].groupby(['公司简称', '股东'])['日期'].apply(list).reset_index()

    # 合并特征数据
    df = df.merge(has_investigated[['公司简称', '股东', 'has_investigated']], 
                on=['公司简称', '股东'], how='left')
    df = df.merge(penalty_dates.rename(columns={'日期': 'penalty_dates'}),
                on=['公司简称', '股东'], how='left')
    
    # 处理空值
    df['has_investigated'] = df['has_investigated'].fillna(False)
    df['penalty_dates'] = df['penalty_dates'].apply(lambda x: x if isinstance(x, list) else [])

    # 条件验证函数
    def check_condition(row):
        if not row['has_investigated']:
            return False
        for penalty_date in row['penalty_dates']:
            if (row['日期'] - penalty_date).days >= 0 and \
               (row['日期'] - penalty_date).days <= 180:
                return True
        return False

    df['meu_10_2_condition'] = df.apply(check_condition, axis=1)

    # 3. 验证约束条件（独立检查）
    df['meu_10_2_constraint'] = df['当日减持比例'] == 0

    # 清理中间列
    df.drop(columns=['has_investigated', 'penalty_dates'], inplace=True, errors='ignore')

    return df

In [ ]:
df = check_meu_10_2(df)
df

---

## MEU_10_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉及与本上市公司有关的违法违规，被本所公开谴责未满3个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1704 |
| completion_tokens | 5748 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_10_3(df):
    '''
    检查MEU_10_3合规性：
    subject: 上市公司大股东
    condition: 因涉及违法违规被本所公开谴责未满3个月
    constraint: 不得减持股份
    '''
    df = df.copy()
    
    # 初始化合规标记列
    df['meu_10_3_subject'] = False
    df['meu_10_3_condition'] = False
    df['meu_10_3_constraint'] = None
    
    # 1. 验证责任主体
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[is_major_shareholder, 'meu_10_3_subject'] = True
    
    # 2. 验证触发条件
    # 获取股东最近公开谴责日期
    condemn_df = df[df['股东涉嫌证券期货违法犯罪事件'] == '被本所公开谴责']
    if not condemn_df.empty:
        last_condemn = condemn_df.groupby(['公司简称', '股东'])['日期'].max().reset_index()
        last_condemn.rename(columns={'日期':'last_condemn_date'}, inplace=True)
        df = df.merge(last_condemn, how='left', on=['公司简称', '股东'])
        # 计算自然日间隔
        df['condemn_days'] = (df['日期'] - df['last_condemn_date']).dt.days
        valid_cond = (df['condemn_days'] <= 90) & df['last_condemn_date'].notna()
        df.loc[valid_cond, 'meu_10_3_condition'] = True
        df.drop(columns=['last_condemn_date', 'condemn_days'], inplace=True)
    else:
        df['meu_10_3_condition'] = False
    
    # 3. 验证约束条件（独立检查所有记录）
    df['meu_10_3_constraint'] = df['当日减持比例'] <= 0
    
    return df

In [ ]:
df = check_meu_10_3(df)
df

---

## MEU_10_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉及证券期货违法，被中国证监会行政处罚且罚没款尚未足额缴纳，且不存在法律、行政法规另有规定或减持资金用于缴纳罚没款的情形 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1723 |
| completion_tokens | 7540 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_10_4(df):
    '''
    验证MEU_10_4合规性：
    subject: 上市公司大股东
    condition: 因证券期货违法被行政处罚且罚没款未缴，且不存在例外情形
    constraint: 不得减持股份
    '''
    df = df.copy()

    # 初始化合规性标记列
    df['meu_10_4_subject'] = False
    df['meu_10_4_condition'] = False
    df['meu_10_4_constraint'] = None

    # 1. 验证责任主体（上市公司大股东）
    major_shareholder_mask = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) | 
        (df['持股比例'] >= 0.05)
    )
    df.loc[major_shareholder_mask, 'meu_10_4_subject'] = True

    # 2. 验证触发条件（行政处罚+未足额缴纳+无例外情形）
    # 按时间顺序计算行政处罚持续状态
    sorted_df = df.sort_values(['公司简称', '股东', '日期'])
    sorted_df['行政处罚持续状态'] = (
        (sorted_df['股东涉嫌证券期货违法犯罪事件'] == '行政处罚决定作出')
        .groupby([sorted_df['公司简称'], sorted_df['股东']])
        .cummax()
    )
    df['行政处罚持续状态'] = sorted_df['行政处罚持续状态'].values  # 保持原始索引顺序
    
    # 检查例外情形（减持资金用于缴纳罚没款）
    exception_mask = df['拟减持原因'] == '缴纳罚没款'
    valid_condition = df['行政处罚持续状态'] & ~exception_mask
    df.loc[valid_condition, 'meu_10_4_condition'] = True

    # 3. 验证约束条件（当日无减持）
    df['meu_10_4_constraint'] = df['当日减持比例'].eq(0)

    # 清理临时字段
    df.drop('行政处罚持续状态', axis=1, inplace=True)
    
    return df

In [ ]:
df = check_meu_10_4(df)
df

---

# Law Article 11

## MEU_11_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司因涉嫌证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1714 |
| completion_tokens | 4512 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_11_1(df):
    '''
    检查MEU_11_1合规性：
    subject: 上市公司控股股东 | 实际控制人
    condition: 公司因涉嫌证券期货违法犯罪被立案调查/侦查期间
    constraint: 不得减持股份
    
    实现逻辑：
    1. subject通过股东身份字段直接匹配
    2. condition通过公司涉嫌违法事件字段状态判断
    3. constraint通过当日减持比例字段数值验证
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_11_1_subject'] = False
    df['meu_11_1_condition'] = False
    df['meu_11_1_constraint'] = None

    # 标记责任主体 (步骤1)
    subject_mask = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[subject_mask, 'meu_11_1_subject'] = True

    # 标记触发条件 (步骤2)
    condition_choices = ['被中国证监会及其派出机构立案调查', '被司法机关立案侦查']
    df['meu_11_1_condition'] = df['公司涉嫌证券期货违法犯罪事件'].isin(condition_choices)

    # 标记约束条件 (步骤3)
    # 当日减持比例>0视为违反约束，注意处理空值
    df['meu_11_1_constraint'] = (df['当日减持比例'].fillna(0) <= 0)
    
    # 将布尔值转换为True/False明确标记
    df['meu_11_1_constraint'] = df['meu_11_1_constraint'].map({True: True, False: False})

    return df

In [ ]:
df = check_meu_11_1(df)
df

---

## MEU_11_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司因涉嫌证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月的 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1725 |
| completion_tokens | 5506 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_11_2(df):
    '''
    验证MEU_11_2合规性：
    subject: 上市公司控股股东 | 实际控制人
    condition: 公司被立案调查/侦查且处于处罚后6个月内
    constraint: 不得减持股份
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_11_2_subject'] = False
    df['meu_11_2_condition'] = False
    df['meu_11_2_constraint'] = None

    # 1. 验证责任主体
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[valid_subject, 'meu_11_2_subject'] = True

    # 2. 验证触发条件
    # 获取公司最近处罚日期
    penalty_mask = df['公司涉嫌证券期货违法犯罪事件'].isin(['行政处罚决定作出', '刑事判决作出'])
    latest_penalty = df[penalty_mask].groupby('公司简称')['日期'].max().reset_index()
    latest_penalty.columns = ['公司简称', 'latest_penalty_date']
    
    # 合并处罚日期并计算时间差
    df = df.merge(latest_penalty, on='公司简称', how='left')
    df['penalty_days'] = (df['日期'] - df['latest_penalty_date']).dt.days
    valid_condition = (df['penalty_days'] >= 0) & (df['penalty_days'] <= 180)
    df.loc[valid_condition, 'meu_11_2_condition'] = True

    # 3. 验证约束条件（独立检查）
    df['meu_11_2_constraint'] = df['当日减持比例'] == 0
    df['meu_11_2_constraint'] = df['meu_11_2_constraint'].astype(bool)

    # 清理中间列
    df.drop(['latest_penalty_date', 'penalty_days'], axis=1, inplace=True)

    return df

In [ ]:
df = check_meu_11_2(df)
df

---

## MEU_11_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司被本所公开谴责未满3个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1703 |
| completion_tokens | 5146 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_11_3(df):
    '''
    验证MEU_11_3合规性：
    "subject": "上市公司控股股东 | 实际控制人", 
    "condition": "上市公司被本所公开谴责未满3个月", 
    "constraint": "不得减持其所持有的本公司股份"
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_11_3_subject'] = False
    df['meu_11_3_condition'] = False
    df['meu_11_3_constraint'] = None

    # 1. 标记valid的subject
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[valid_subject, 'meu_11_3_subject'] = True

    # 2. 标记valid的condition
    # 获取各公司被公开谴责的日期列表
    condemn_dates = (
        df[df['公司涉嫌证券期货违法犯罪事件'] == '被本所公开谴责']
        .groupby('公司简称')['日期']
        .apply(list)
        .to_dict()
    )

    # 定义日期检查函数
    def check_condemn_window(row):
        company = row['公司简称']
        current_date = row['日期']
        for d in condemn_dates.get(company, []):
            if 0 <= (current_date - d).days <= 90:
                return True
        return False

    df['meu_11_3_condition'] = df.apply(check_condemn_window, axis=1)

    # 3. 标记valid的constraint（独立检查）
    # 通过当日减持比例判断是否发生减持
    valid_constraint = df['当日减持比例'] <= 0
    df.loc[valid_constraint, 'meu_11_3_constraint'] = True
    df.loc[~valid_constraint, 'meu_11_3_constraint'] = False

    return df

In [ ]:
df = check_meu_11_3(df)
df

---

## MEU_11_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 市公司股票因可能触及重大违法强制退市情形，而被本所实施退市风险警示，在本所规定的限制减持期限内的 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1722 |
| completion_tokens | 7246 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_11_4(df):
    '''
    检查MEU_11_4合规性：
    subject: 控股股东或实际控制人
    condition: 公司因重大违法被实施退市风险警示且在限制期内
    constraint: 不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_11_4_subject'] = False
    df['meu_11_4_condition'] = False
    df['meu_11_4_constraint'] = None

    # 标记valid的subject
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[valid_subject, 'meu_11_4_subject'] = True

    # 标记valid的condition（需要分组处理公司状态）
    def get_restriction_periods(group):
        '''识别公司退市风险警示期间'''
        periods = []
        in_restriction = False
        start_date = None
        
        # 按时间排序处理事件序列
        sorted_group = group.sort_values('日期')
        for _, row in sorted_group.iterrows():
            event = row['公司涉嫌证券期货违法犯罪事件']
            current_date = row['日期']
            
            if event == '被中国证监会及其派出机构立案调查' and not in_restriction:
                start_date = current_date
                in_restriction = True
            elif event in ['中国证监会及其派出机构立案调查结束', '行政处罚决定作出'] and in_restriction:
                end_date = current_date
                periods.append((start_date, end_date))
                in_restriction = False
                start_date = None
        
        # 处理未结束的警示状态
        if in_restriction and start_date:
            end_date = sorted_group['日期'].max()
            periods.append((start_date, end_date))
            
        return periods

    # 获取各公司限制期列表
    restriction_dict = df.groupby('公司简称').apply(get_restriction_periods).to_dict()

    # 检查日期是否在限制期内
    def check_condition(row):
        company = row['公司简称']
        date = row['日期']
        for period in restriction_dict.get(company, []):
            if period[0] <= date <= period[1]:
                return True
        return False

    valid_condition = df.apply(check_condition, axis=1)
    df.loc[valid_condition, 'meu_11_4_condition'] = True

    # 标记valid的constraint（独立检查所有记录）
    df['meu_11_4_constraint'] = (df['当日减持比例'] == 0)

    return df

In [ ]:
df = check_meu_11_4(df)
df

---

# Law Article 12

## MEU_12_1


| 字段 | 内容 |
|------|------|
| subject | 公开发行股票并上市时的控股股东 | 公开发行股票并上市时的实际控制人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1778 |
| completion_tokens | 6948 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_1(df):
    '''
    检查MEU_12_1合规性：
    subject: 公开发行股票并上市时的控股股东 | 实际控制人
    condition: 计划通过集中竞价或大宗交易减持股份且首次披露减持计划
    constraint: 最近20个交易日内任一交易日复权收盘价不低于发行价
    contextual_info: 复权计算以发行日为准
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_12_1_subject'] = False
    df['meu_12_1_condition'] = False
    df['meu_12_1_constraint'] = None

    # 处理subject部分
    # 获取各公司上市日期（首个非空收盘价日期）
    def get_ipo_date(group):
        valid_dates = group[group['收盘价'].notna()]['日期']
        return valid_dates.min() if not valid_dates.empty else pd.NaT
    company_ipo_dates = df.groupby('公司简称').apply(get_ipo_date).reset_index(name='公司上市日期')
    
    # 合并上市日期并筛选上市当日股东身份
    df = df.merge(company_ipo_dates, on='公司简称', how='left')
    ipo_day_mask = (df['日期'] == df['公司上市日期']) & df['股东身份'].isin(['控股股东', '实际控制人'])
    valid_subjects = df[ipo_day_mask][['公司简称', '股东']].drop_duplicates()
    
    # 标记subject合规
    df['meu_12_1_subject'] = df.apply(
        lambda x: (x['公司简称'], x['股东']) in valid_subjects.values, 
        axis=1
    )

    # 处理condition部分
    # 获取首次减持计划披露日
    has_plan = df[df['存在减持计划']]
    first_disclosure = has_plan.groupby(['公司简称', '股东'])['日期'].min().reset_index(name='首次披露日')
    
    # 合并首次披露日并验证条件
    df = df.merge(first_disclosure, on=['公司简称', '股东'], how='left')
    valid_condition = (
        df['减持方式'].isin(['竞价交易', '大宗交易']) & 
        df['存在减持计划'] & 
        (df['日期'] == df['首次披露日'])
    )
    df['meu_12_1_condition'] = valid_condition

    # 处理constraint部分
    # 计算复权价格并标记价格违规
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    df['价格违规'] = df['复权收盘价'] < df['发行价格']
    
    # 按公司滚动检查20日窗口
    def check_price_violation(group):
        group = group.sort_values('日期')
        group['20日违规标记'] = group['价格违规'].rolling(20, min_periods=1).apply(any, raw=True)
        return group
    
    df = df.groupby('公司简称', group_keys=False).apply(check_price_violation)
    df['meu_12_1_constraint'] = ~df['20日违规标记']

    # 清理中间列
    df.drop([
        '公司上市日期', '首次披露日', 
        '复权收盘价', '价格违规', '20日违规标记'
    ], axis=1, errors='ignore', inplace=True)

    return df

In [ ]:
df = check_meu_12_1(df)
df

---

## MEU_12_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 | 一致行动人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：最近20个交易日内，上市公司任一交易日股票收盘价低于最近一个会计年度或者最近一期财务会计报告期末每股归属于上市公司股东的净资产 |
| contextual_info | 股票收盘价以最近一个会计年度或者最近一期财务会计报告资产负债表日为基准分别向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1787 |
| completion_tokens | 6694 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_2(df):
    '''
    验证MEU_12_2合规性：
    subject: 上市公司控股股东 | 实际控制人 | 一致行动人
    condition: 计划通过集中竞价或大宗交易减持且首次披露计划
    constraint: 最近20个交易日内收盘价不低于最近财务报告每股净资产
    
    处理逻辑：
    1. subject验证控股股东和实际控制人（一致行动人无数据支持）
    2. condition验证交易方式、减持计划及首次披露
    3. constraint验证复权后股价与净资产关系
    '''
    df = df.copy()
    
    # 初始化结果列
    df['meu_12_2_subject'] = False
    df['meu_12_2_condition'] = False
    df['meu_12_2_constraint'] = None
    
    # 1. SUBJECT验证（控股股东/实际控制人）
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[valid_subject, 'meu_12_2_subject'] = True
    
    # 2. CONDITION验证
    # 交易方式验证
    valid_trade_type = df['减持方式'].isin(['竞价交易', '大宗交易'])
    # 首次披露验证（按股东维度）
    df_sorted = df.sort_values(by='日期')
    first_disclosure = (
        df_sorted.groupby(['公司简称', '股东'], group_keys=False)
        ['存在减持计划'].transform(lambda x: x & (x.cumsum() == 1))
    )
    valid_condition = valid_trade_type & df['存在减持计划'] & first_disclosure
    df.loc[valid_condition, 'meu_12_2_condition'] = True
    
    # 3. CONSTRAINT验证
    # 计算复权价格并判断
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    df['price_below_na'] = df['复权收盘价'] < df['每股净资产']
    
    # 按公司维度滚动检查20交易日
    df_sorted = df.sort_values(by='日期')
    window_check = (
        df_sorted.groupby('公司简称', group_keys=False)
        ['price_below_na'].rolling(window=20, min_periods=1)
        .apply(lambda x: x.any()).astype(bool)
    )
    df['constraint_violated'] = window_check.reset_index(level=0, drop=True)
    df['meu_12_2_constraint'] = ~df['constraint_violated']
    
    # 清理中间列
    df.drop(['复权收盘价', 'price_below_na', 'constraint_violated'], 
            axis=1, inplace=True)
    
    return df

In [ ]:
df = check_meu_12_2(df)
df

---

## MEU_12_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 | 一致行动人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：上市公司最近一期经审计的财务报告的归属于上市公司股东的净利润为负 |
| contextual_info | nan |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1752 |
| completion_tokens | 3516 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_3(df):
    '''
    检查MEU_12_3合规性：
    subject: 上市公司控股股东 | 实际控制人 | 一致行动人
    condition: 计划通过集中竞价或大宗交易减持股份且首次披露减持计划
    constraint: 最近一期经审计净利润不得为负
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_12_3_subject'] = False
    df['meu_12_3_condition'] = False
    df['meu_12_3_constraint'] = None

    # 1. 标记valid的subject
    # 处理控股股东和实际控制人（数据中无一致行动人字段）
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[valid_subject, 'meu_12_3_subject'] = True

    # 2. 标记valid的condition
    # 按公司和股东分组处理首次披露
    df_sorted = df.sort_values(['公司简称', '股东', '日期'])
    grouped = df_sorted.groupby(['公司简称', '股东'])
    # 计算历史减持计划累计次数（不含当前行）
    df_sorted['prev_plan_count'] = grouped['存在减持计划'].cumsum() - df_sorted['存在减持计划']
    df = df_sorted.sort_index()

    # 条件验证
    valid_condition = (
        df['减持方式'].isin(['竞价交易', '大宗交易']) &  # 交易方式验证
        df['存在减持计划'] &  # 存在减持计划
        (df['prev_plan_count'] == 0)  # 首次披露验证
    )
    df.loc[valid_condition, 'meu_12_3_condition'] = True

    # 3. 标记valid的constraint（独立检查）
    # 净利润>=0时满足约束
    df['meu_12_3_constraint'] = df['净利润'] >= 0

    # 清理临时列
    df.drop('prev_plan_count', axis=1, inplace=True, errors='ignore')

    return df

In [ ]:
df = check_meu_12_3(df)
df

---

## MEU_12_4


| 字段 | 内容 |
|------|------|
| subject | 控股股东 | 实际控制人（上市后不再具备相关主体身份的） |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1775 |
| completion_tokens | 7249 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_4(df):
    '''
    验证MEU_12_4合规性：
    subject: 上市时是控股股东/实际控制人且当前已不具备该身份
    condition: 首次披露竞价/大宗减持计划
    constraint: 最近20交易日无复权收盘价低于发行价
    '''
    df = df.copy()
    
    # 初始化结果列
    df['meu_12_4_subject'] = False
    df['meu_12_4_condition'] = False
    df['meu_12_4_constraint'] = False
    
    # ================== SUBJECT验证 ==================
    # 构建公司上市时控股股东/实际控制人字典
    ipo_entities = (df.sort_values('日期')
                    .groupby('公司简称')
                    .apply(lambda g: set(g[g['日期'] == g['上市日期'].iloc[0]]
                                        .query("股东身份 in ['控股股东','实际控制人']")['股东'])))
    
    # 标记符合主体条件
    df['is_ipo_entity'] = df.apply(
        lambda x: x['股东'] in ipo_entities.get(x['公司简称'], set()), axis=1)
    valid_subject = df['is_ipo_entity'] & ~df['股东身份'].isin(['控股股东','实际控制人'])
    df.loc[valid_subject, 'meu_12_4_subject'] = True

    # ================== CONDITION验证 ==================
    # 计算首次计划披露日期
    first_plan = (df[df['存在减持计划']]
                 .sort_values('日期')
                 .groupby(['公司简称','股东'])['日期']
                 .first()
                 .reset_index(name='首披日'))
    
    # 合并首披日并验证条件
    df = df.merge(first_plan, how='left')
    valid_condition = (
        df['存在减持计划'] & 
        (df['日期'] == df['首披日']) & 
        df['减持方式'].isin(['竞价交易','大宗交易'])
    )
    df.loc[valid_condition, 'meu_12_4_condition'] = True

    # ================== CONSTRAINT验证 ==================
    # 计算复权价格
    df['复权价'] = df['收盘价'] * df['复权因子']
    
    # 按公司计算滚动窗口内最低价
    df_sorted = df.sort_values(['公司简称','日期'])
    df_sorted['发行价'] = df_sorted.groupby('公司简称')['发行价格'].transform('first')
    window_check = df_sorted.groupby('公司简称', group_keys=False).apply(
        lambda g: g['复权价'].rolling(20, min_periods=1).apply(
            lambda x: (x < g['发行价'].iloc[0]).any()))
    
    # 标记约束违规
    df_sorted['meu_12_4_constraint'] = ~(window_check > 0)
    df = df_sorted.sort_index().drop(columns=['复权价','发行价'])
    
    return df.drop(columns=['is_ipo_entity','首披日'], errors='ignore')

In [ ]:
df = check_meu_12_4(df)
df

---

## MEU_12_5


| 字段 | 内容 |
|------|------|
| subject | 公开发行时持股5%以上的第一大股东 | 一致行动人 |
| condition | 上市公司在公开发行股票并上市时披露为无控股股东、实际控制人, 且计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1793 |
| completion_tokens | 6905 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_12_5(df):
    df = df.copy()
    
    # 初始化标记列
    df['meu_12_5_subject'] = False
    df['meu_12_5_condition'] = False
    df['meu_12_5_constraint'] = None
    
    # ================== SUBJECT CHECK ==================
    # 验证公开发行时持股5%以上的第一大股东
    # 1. 获取各公司上市日期和发行价格
    company_ipo_info = df.groupby('公司简称').agg(
        上市日期=('上市日期', 'first'),
        发行价格=('发行价格', 'first')
    ).reset_index()
    
    # 2. 找出各公司上市时的第一大股东
    major_shareholders = {}
    for _, row in company_ipo_info.iterrows():
        company = row['公司简称']
        ipo_date = row['上市日期']
        
        # 获取上市当天的股东数据
        ipo_day_data = df[(df['公司简称'] == company) & (df['日期'] == ipo_date)]
        
        if not ipo_day_data.empty:
            # 找到最大持股比例的股东
            max_ratio = ipo_day_data['持股比例'].max()
            if max_ratio >= 0.05:
                valid_shareholders = ipo_day_data[ipo_day_data['持股比例'] == max_ratio]['股东'].unique()
                major_shareholders[company] = list(valid_shareholders)
            else:
                major_shareholders[company] = []
    
    # 3. 标记有效subject
    df['meu_12_5_subject'] = df.apply(
        lambda x: x['股东'] in major_shareholders.get(x['公司简称'], []),
        axis=1
    )
    
    # ================== CONDITION CHECK ==================
    # 验证上市公司无实际控制人且首次披露减持计划
    # 1. 检查公司是否无实际控制人
    df['company_no_control'] = df.groupby('公司简称')['股东身份'].transform(
        lambda x: ~x.isin(['控股股东', '实际控制人']).any()
    )
    
    # 2. 验证减持方式
    valid_method = df['减持方式'].isin(['竞价交易', '大宗交易'])
    
    # 3. 首次减持计划标记
    df.sort_values(['公司简称', '股东', '日期'], inplace=True)
    df['plan_seq'] = df.groupby(['公司简称', '股东'])['存在减持计划'].cumsum()
    first_plan = (df['存在减持计划']) & (df['plan_seq'] == 1)
    
    # 组合条件
    df['meu_12_5_condition'] = df['company_no_control'] & valid_method & first_plan
    
    # ================== CONSTRAINT CHECK ==================
    # 验证20日收盘价不低于发行价
    # 1. 计算复权价格
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    
    # 2. 获取发行价格
    df = df.merge(company_ipo_info[['公司简称', '发行价格']], on='公司简称', how='left')
    
    # 3. 滚动检查最近20个交易日
    df.sort_values(['公司简称', '日期'], inplace=True)
    df['price_valid'] = df.groupby('公司简称').apply(
        lambda g: g['复权收盘价'].rolling(20, min_periods=1).min() >= g['发行价格']
    ).reset_index(level=0, drop=True)
    
    df['meu_12_5_constraint'] = df['price_valid']
    
    # 清理中间列
    df.drop(columns=['company_no_control', 'plan_seq', '复权收盘价', '发行价格', 'price_valid'],
           inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_meu_12_5(df)
df

---

# Law Article 13

## MEU_13_2


| 字段 | 内容 |
|------|------|
| subject | 大股东 |
| condition | 通过协议转让方式减持股份导致出让方不再具有大股东身份, 且处在减持后6个月内 |
| constrain | 继续遵守本指引第四条规定 |
| contextual_info | nan |
| note | nan |
| relation | should_include |
| target | Law_4 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1709 |
| completion_tokens | 7409 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_13_2(df):
    '''
    检查MEU_13_2合规性：
    subject: 大股东（控股股东/实际控制人/持股5%以上股东或持股比例≥5%）
    condition: 通过协议转让减持导致失去大股东身份且在减持后6个月内
    constraint: 继续遵守第四条（假设为减持预披露要求）
    '''
    df = df.copy()
    
    # 初始化标记列
    df['meu_13_2_subject'] = False
    df['meu_13_2_condition'] = False
    df['meu_13_2_constraint'] = False

    # 1. 标记subject有效性
    major_roles = ['控股股东', '实际控制人', '持股5%以上股东']
    df['meu_13_2_subject'] = df['股东身份'].isin(major_roles) | (df['持股比例'] >= 0.05)

    # 2. 标记condition有效性（分组处理股东减持事件）
    for (company, shareholder), group in df.groupby(['公司简称', '股东']):
        # 找出协议转让且导致失去大股东资格的日期
        condition_dates = group[
            (group['减持方式'] == '协议转让') &
            (group['持股比例'] < 0.05)  # 减持后失去大股东身份
        ]['日期']
        
        # 标记后续6个月内的交易日
        for trigger_date in condition_dates:
            mask = (
                (df['公司简称'] == company) &
                (df['股东'] == shareholder) &
                (df['日期'].between(
                    trigger_date,
                    trigger_date + pd.DateOffset(months=6) - pd.DateOffset(days=1),
                    inclusive='both'
                ))
            )
            df.loc[mask, 'meu_13_2_condition'] = True

    # 3. 标记constraint有效性（独立检查所有减持计划的预披露）
    df['预披露天数'] = (df['计划开始日'] - df['计划披露日']).dt.days
    df['meu_13_2_constraint'] = df['存在减持计划'] & (df['预披露天数'] >= 15)

    return df

In [ ]:
df = check_meu_13_2(df)
df

---

## MEU_13_3


| 字段 | 内容 |
|------|------|
| subject | 控股股东 | 实际控制人 |
| condition | 通过协议转让方式减持股份导致其不再具有控股股东、实际控制人身份, 且处于减持后的6个月内 |
| constrain | 应当继续遵守本指引第十二条第一款第二、三项规定 |
| contextual_info | nan |
| note | nan |
| relation | should_include |
| target | MEU_12_1;MEU_12_2 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1723 |
| completion_tokens | 9385 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_13_3(df):
    '''
    检查MEU_13_3的合规性：
    subject: 控股股东 | 实际控制人
    condition: 通过协议转让减持导致失去身份，且在减持后的6个月内
    constraint: 继续遵守第十二条相关规定（假设检查减持后的六个月内是否有减持行为）
    '''
    df = df.copy()
    
    # 初始化标记列
    df['meu_13_3_subject'] = df['股东身份'].isin(['控股股东', '实际控制人'])
    df['meu_13_3_condition'] = False
    df['meu_13_3_constraint'] = True  # 初始假设为True
    
    # 处理condition和constraint
    # 分组处理每个公司和股东
    def process_group(group):
        # 找到所有协议转让且subject为True的行
        protocol_mask = (group['减持方式'] == '协议转让') & (group['meu_13_3_subject'])
        protocol_dates = group.loc[protocol_mask, '日期'].tolist()
        
        # 检查每个协议转让日期是否导致身份变化
        for date in protocol_dates:
            # 找到该日期之后的行
            subsequent = group[group['日期'] > date]
            if not subsequent.empty:
                # 检查后续第一个行是否身份变化
                first_subsequent = subsequent.iloc[0]
                if not first_subsequent['meu_13_3_subject']:
                    # 触发点date，计算六个月后的日期
                    end_date = date + pd.DateOffset(months=6)
                    # 标记condition为True
                    condition_mask = (group['日期'] >= date) & (group['日期'] <= end_date)
                    group.loc[condition_mask, 'meu_13_3_condition'] = True
        
        # 独立处理constraint：检查所有减持后的六个月内是否有减持行为
        dissolution_dates = group[group['当日减持比例'] > 0]['日期']
        for date in dissolution_dates:
            end_date = date + pd.DateOffset(months=6)
            constraint_mask = (group['日期'] > date) & (group['日期'] <= end_date)
            group.loc[constraint_mask, 'meu_13_3_constraint'] = False
        
        return group
    
    df = df.groupby(['公司简称', '股东'], group_keys=False).apply(process_group)
    
    return df

In [ ]:
df = check_meu_13_3(df)
df

---

# Law Article 14

## MEU_14_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司因涉嫌证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1710 |
| completion_tokens | 4302 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_1(df):
    '''
    检查MEU_14_1合规性：
    subject: 上市公司董监高
    condition: 公司因涉嫌证券期货违法犯罪被立案调查或侦查期间
    constraint: 不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_14_1_subject'] = False
    df['meu_14_1_condition'] = False
    df['meu_14_1_constraint'] = None

    # 1. 验证责任主体：股东身份为董监高
    valid_subject = df['股东身份'] == '董监高'
    df.loc[valid_subject, 'meu_14_1_subject'] = True

    # 2. 验证触发条件：公司处于被立案调查/侦查状态
    investigation_conditions = ['被中国证监会及其派出机构立案调查', '被司法机关立案侦查']
    valid_condition = df['公司涉嫌证券期货违法犯罪事件'].isin(investigation_conditions)
    df.loc[valid_condition, 'meu_14_1_condition'] = True

    # 3. 验证约束条件：当日无减持行为
    # 处理空值并转换为数值型
    df['当日减持比例'] = pd.to_numeric(df['当日减持比例'], errors='coerce').fillna(0)
    valid_constraint = df['当日减持比例'] == 0
    # 设置约束标记
    df.loc[valid_constraint, 'meu_14_1_constraint'] = True
    df.loc[~valid_constraint, 'meu_14_1_constraint'] = False

    return df

In [ ]:
df = check_meu_14_1(df)
df

---

## MEU_14_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司因涉嫌证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1721 |
| completion_tokens | 5117 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_2(df):
    '''
    合规性检查函数：MEU_14_2
    subject: 上市公司董监高
    condition: 公司因证券期货违法犯罪被立案调查/侦查且处罚决定/刑事判决未满6个月
    constraint: 不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_14_2_subject'] = False
    df['meu_14_2_condition'] = False
    df['meu_14_2_constraint'] = None

    # 1. 标记责任主体 (上市公司董监高)
    subject_mask = df['股东身份'] == '董监高'
    df.loc[subject_mask, 'meu_14_2_subject'] = True

    # 2. 标记触发条件 (处罚后未满6个月)
    # 筛选处罚相关事件
    penalty_events = df[df['公司涉嫌证券期货违法犯罪事件'].isin(['行政处罚决定作出', '刑事判决作出'])]
    
    # 按公司获取最新处罚日期
    latest_penalty = penalty_events.groupby('公司简称')['公告日期'].max().reset_index()
    latest_penalty.rename(columns={'公告日期':'latest_penalty_date'}, inplace=True)
    
    # 合并处罚日期到主表
    df = df.merge(latest_penalty, on='公司简称', how='left')
    
    # 计算自然日间隔
    if 'latest_penalty_date' in df.columns:
        df['penalty_days'] = (df['日期'] - df['latest_penalty_date']).dt.days
        condition_mask = (df['penalty_days'] >= 0) & (df['penalty_days'] <= 180)
        df.loc[condition_mask, 'meu_14_2_condition'] = True

    # 3. 标记约束条件 (不得减持)
    df['meu_14_2_constraint'] = df['当日减持比例'] <= 0  # 无减持为合规
    
    # 清理辅助列
    df.drop(['latest_penalty_date', 'penalty_days'], axis=1, errors='ignore', inplace=True)
    
    return df

In [ ]:
df = check_meu_14_2(df)
df

---

## MEU_14_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉嫌与该上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1713 |
| completion_tokens | 3719 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_3(df):
    '''
    验证MEU_14_3合规性：
    subject: 上市公司董监高
    condition: 因涉嫌证券期货违法犯罪被立案调查/侦查期间
    constraint: 不得减持股份
    '''
    df = df.copy()

    # 初始化合规性标记列
    df['meu_14_3_subject'] = False
    df['meu_14_3_condition'] = False
    df['meu_14_3_constraint'] = None

    # 1. 验证责任主体（当前在任的董监高）
    valid_subject = df['股东身份'] == '董监高'
    df.loc[valid_subject, 'meu_14_3_subject'] = True

    # 2. 验证触发条件（处于被立案调查/侦查状态）
    investigation_status = ['被中国证监会及其派出机构立案调查', '被司法机关立案侦查']
    valid_condition = df['股东涉嫌证券期货违法犯罪事件'].isin(investigation_status)
    df.loc[valid_condition, 'meu_14_3_condition'] = True

    # 3. 验证约束条件（当日未发生减持行为）
    # 注意：此处独立检查，不考虑subject和condition是否满足
    valid_constraint = (df['当日减持比例'] <= 0)
    df.loc[valid_constraint, 'meu_14_3_constraint'] = True
    df.loc[~valid_constraint, 'meu_14_3_constraint'] = False

    return df

In [ ]:
df = check_meu_14_3(df)
df

---

## MEU_14_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉嫌与该上市公司有关的证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1724 |
| completion_tokens | 6450 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_4(df):
    '''
    检查MEU_14_4合规性：
    subject: 上市公司董监高
    condition: 因涉嫌证券期货违法犯罪被立案且处罚后未满6个月
    constraint: 不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_14_4_subject'] = False
    df['meu_14_4_condition'] = False
    df['meu_14_4_constrain'] = True  # 默认未减持

    # 1.标记subject(上市公司董监高)
    df['meu_14_4_subject'] = df['股东身份'].isin(['董监高'])

    # 2.标记condition(被立案且处罚后未满6个月)
    # 按股东分组处理
    grouped = df.groupby(['公司简称', '股东'], group_keys=False)
    
    # 判断是否被立案调查/侦查过
    df['_investigated'] = grouped['股东涉嫌证券期货违法犯罪事件'].transform(
        lambda x: x.isin(['被中国证监会及其派出机构立案调查', '被司法机关立案侦查']).any()
    )
    
    # 获取最新行政处罚日期
    penalty_dates = df[df['股东涉嫌证券期货违法犯罪事件'] == '行政处罚决定作出']\
        .groupby(['公司简称', '股东'])['日期'].max().rename('penalty_date')
    df = df.merge(penalty_dates, how='left', on=['公司简称', '股东'])
    
    # 计算自然日间隔
    df['_penalty_days'] = (df['日期'] - df['penalty_date']).dt.days
    valid_penalty = (df['_penalty_days'] >= 0) & (df['_penalty_days'] <= 180)
    
    # 综合条件判断
    df['meu_14_4_condition'] = df['_investigated'] & valid_penalty & df['penalty_date'].notna()

    # 3.标记constraint(当日无减持)
    df['meu_14_4_constrain'] = (df['当日减持比例'] <= 0)

    # 清理中间列
    df.drop(['_investigated', 'penalty_date', '_penalty_days'], axis=1, inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_meu_14_4(df)
df

---

## MEU_14_5


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉及证券期货违法，被中国证监会行政处罚，罚没款尚未足额缴纳，且不存在法律、行政法规另有规定或减持资金用于缴纳罚没款的情况 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 不考虑法律、行政法规另有规定或减持资金用于缴纳罚没款的情况 |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1740 |
| completion_tokens | 4183 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_5(df):
    '''
    检查MEU_14_5合规性：
    "subject": "上市公司董监高",
    "condition": "被证监会行政处罚且罚没款未缴",
    "constraint": "不得减持股份",
    实现逻辑：
    1. subject检查：股东身份为董监高
    2. condition检查：存在行政处罚决定且未缴款（通过事件字段判断）
    3. constraint检查：当日无减持行为（通过减持比例判断）
    '''
    df = df.copy()

    # 初始化合规标记列
    df['meu_14_5_subject'] = False
    df['meu_14_5_condition'] = False
    df['meu_14_5_constraint'] = None

    # 1. 验证责任主体（上市公司董监高）
    subject_mask = df['股东身份'] == '董监高'
    df.loc[subject_mask, 'meu_14_5_subject'] = True

    # 2. 验证触发条件（存在未缴款的行政处罚）
    # 根据数据字段直接匹配行政处罚决定状态
    condition_mask = df['股东涉嫌证券期货违法犯罪事件'] == '行政处罚决定作出'
    df.loc[condition_mask, 'meu_14_5_condition'] = True

    # 3. 验证约束条件（未发生减持行为）
    # 通过当日减持比例判断，0表示无减持
    constraint_mask = df['当日减持比例'] == 0
    df.loc[constraint_mask, 'meu_14_5_constraint'] = True
    df.loc[~constraint_mask, 'meu_14_5_constraint'] = False

    return df

In [ ]:
df = check_meu_14_5(df)
df

---

## MEU_14_6


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉及与本上市公司有关的违法违规，被证券交易所公开谴责未满三个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1704 |
| completion_tokens | 5533 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_6(df):
    '''
    检查MEU_14_6合规性：
    subject: 上市公司董监高
    condition: 被交易所公开谴责未满三个月
    constraint: 不得减持股份
    
    处理逻辑：
    1. subject标记：股东身份为"董监高"
    2. condition标记：按股东追踪最近谴责日期，计算90天窗口期
    3. constraint标记：当日无减持行为（减持比例<=0）
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_14_6_subject'] = False
    df['meu_14_6_condition'] = False
    df['meu_14_6_constraint'] = None

    # 标记valid的subject
    df['meu_14_6_subject'] = df['股东身份'] == '董监高'

    # 标记valid的condition
    # 创建临时谴责日期列
    df['_condemn_date'] = pd.NaT
    condemn_mask = df['股东涉嫌证券期货违法犯罪事件'] == '被本所公开谴责'
    df.loc[condemn_mask, '_condemn_date'] = df.loc[condemn_mask, '日期']
    
    # 按股东计算累积最大谴责日期
    df['_last_condemn'] = df.groupby(['公司简称', '股东'])['_condemn_date'].cummax()
    
    # 计算自然日差
    date_diff = df['日期'] - df['_last_condemn']
    valid_condition = (date_diff <= pd.Timedelta(days=90)) & df['_last_condemn'].notna()
    df['meu_14_6_condition'] = valid_condition

    # 标记valid的constraint（独立检查）
    df['meu_14_6_constraint'] = (df['当日减持比例'] <= 0) | df['当日减持比例'].isna()
    df['meu_14_6_constraint'] = df['meu_14_6_constraint'].astype(bool)

    # 清理临时列
    df.drop(columns=['_condemn_date', '_last_condemn'], inplace=True)

    return df

In [ ]:
df = check_meu_14_6(df)
df

---

## MEU_14_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司股票因可能触及重大违法强制退市情形而被本所实施退市风险警示，且处于本所规定的限制转让的期限内 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1718 |
| completion_tokens | 8122 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_14_7(df):
    '''
    检查MEU_14_7合规性：
    subject: 上市公司董监高
    condition: 公司股票因重大违法强制退市风险被实施退市风险警示且处于限制转让期
    constraint: 不得减持所持股份
    
    实现逻辑：
    1. subject标记：股东身份为"董监高"
    2. condition标记：公司存在重大违法调查且处于限制转让期
    3. constraint标记：当日无减持行为
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_14_7_subject'] = False
    df['meu_14_7_condition'] = False
    df['meu_14_7_constraint'] = None

    # 标记subject：股东身份为董监高
    valid_subject = df['股东身份'] == '董监高'
    df.loc[valid_subject, 'meu_14_7_subject'] = True

    # 标记condition：公司处于限制转让期
    def _calc_restriction(group):
        # 获取公司所有重大违法调查日期
        investigation_dates = group[group['公司涉嫌证券期货违法犯罪事件'] == '被中国证监会及其派出机构立案调查']['日期'].unique()
        # 生成限制转让期(假设调查后180自然日)
        restriction_periods = [
            (date, date + pd.Timedelta(days=180)) 
            for date in investigation_dates
        ]
        # 标记当前日期是否在任意限制期内
        group['in_restriction'] = group['日期'].apply(
            lambda x: any(start <= x <= end for start, end in restriction_periods)
        )
        return group
    
    df = df.groupby('公司简称', group_keys=False).apply(_calc_restriction)
    valid_condition = df['in_restriction']
    df.loc[valid_condition, 'meu_14_7_condition'] = True

    # 标记constraint：当日无减持
    valid_constraint = df['当日减持比例'] == 0
    df.loc[valid_constraint, 'meu_14_7_constraint'] = True
    df.loc[~valid_constraint, 'meu_14_7_constraint'] = False

    # 清理辅助列
    df.drop(columns=['in_restriction'], inplace=True, errors='ignore')

    return df

In [ ]:
df = check_meu_14_7(df)
df

---

# Law Article 16

## MEU_16_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司年度报告、半年度报告公告前15日内 |
| constrain | 不得买卖本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1698 |
| completion_tokens | 4784 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_16_1(df):
    '''
    检查MEU_16_1合规性：
    - subject: 上市公司董监高
    - condition: 处于年报/半年报公告前15日内
    - constraint: 当日无股份买卖行为
    '''
    df = df.copy()
    
    # 初始化标记列
    df['meu_16_1_subject'] = False
    df['meu_16_1_condition'] = False
    df['meu_16_1_constraint'] = None

    # 1. 验证责任主体（独立检查）
    valid_subject = df['股东身份'] == '董监高'
    df.loc[valid_subject, 'meu_16_1_subject'] = True

    # 2. 验证触发条件（按公司分组检查）
    # 构建公司报告日期字典（年报/半年报）
    report_dates = df[df['公告类型'].isin(['年度报告', '半年度报告'])]
    company_report_dates = report_dates.groupby('公司简称')['公告日期'].apply(lambda x: x.unique().tolist()).to_dict()
    
    # 定义自然日窗口检查函数
    def check_report_window(row):
        company = row['公司简称']
        current_date = row['日期']
        for report_date in company_report_dates.get(company, []):
            if (report_date - pd.Timedelta(days=15)) <= current_date < report_date:
                return True
        return False
    
    valid_condition = df.apply(check_report_window, axis=1)
    df.loc[valid_condition, 'meu_16_1_condition'] = True

    # 3. 验证约束内容（独立检查）
    # 通过当日减持比例判断是否发生交易
    valid_constraint = df['当日减持比例'] == 0
    df.loc[valid_constraint, 'meu_16_1_constraint'] = True
    df.loc[~valid_constraint, 'meu_16_1_constraint'] = False

    return df

In [ ]:
df = check_meu_16_1(df)
df

---

## MEU_16_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司季度报告、业绩预告、业绩快报公告前5日内 |
| constrain | 不得买卖本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1701 |
| completion_tokens | 6500 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_16_2(df):
    '''
    检查MEU_16_2合规性：
    subject: 上市公司董监高
    condition: 处于季度报告/业绩预告/业绩快报公告前5日内
    constraint: 不得买卖本公司股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['meu_16_2_subject'] = False
    df['meu_16_2_condition'] = False
    df['meu_16_2_constraint'] = False
    
    # 标记valid的subject（董监高）
    valid_subject = df['股东身份'] == '董监高'
    df.loc[valid_subject, 'meu_16_2_subject'] = True
    
    # 标记valid的condition（公告前5日）
    # 获取有效公告日期
    relevant_ann = df[
        df['公告类型'].isin(['季度报告', '业绩预告', '业绩快报']) 
        & df['公告日期'].notna()
    ]
    ann_dates = relevant_ann[['公司简称', '公告日期']].drop_duplicates()
    
    # 生成所有前5日日期集合
    pre_dates_set = set()
    for _, row in ann_dates.iterrows():
        ann_date = row['公告日期']
        for days_back in range(5, 0, -1):
            pre_date = ann_date - pd.Timedelta(days=days_back)
            pre_dates_set.add((row['公司简称'], pre_date))
    
    # 标记符合条件的日期
    df['meu_16_2_condition'] = df.apply(
        lambda x: (x['公司简称'], x['日期']) in pre_dates_set,
        axis=1
    )
    
    # 标记valid的constraint（无减持）
    valid_constraint = df['当日减持比例'] == 0
    df.loc[valid_constraint, 'meu_16_2_constraint'] = True
    
    return df

In [ ]:
df = check_meu_16_2(df)
df

---

# Law Article 17

## MEU_17_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 在其就任时确定的任期内和任期届满后6个月内，每年通过集中竞价、大宗交易、协议转让等方式转让股份且不属于因司法强制执行、继承、遗赠、依法分割财产等导致股份变动 |
| constrain | 每年转让的股份不得超过其所持本公司股份总数的25% |
| contextual_info | nan |
| note | nan |
| relation | refer_to |
| target | MEU_18_1;MEU_18_2;MEU_18_3;MEU_18_4 |
| type | 实际执行单元 |
| comments | 不考虑因司法强制执行、继承、遗赠、依法分割财产等导致股份变动的情况 |
| prompt_tokens | 1821 |
| completion_tokens | 7230 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_17_1(df):
    '''
    验证MEU_17_1合规性：
    subject: 上市公司董监高
    condition: 在任期内及离任后6个月内通过指定方式转让股份且非特殊原因
    constraint: 年转让量不超过上年末持股的25%
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_17_1_subject'] = False
    df['meu_17_1_condition'] = False
    df['meu_17_1_constraint'] = False

    # 标记subject
    valid_subject = df['股东身份'] == '董监高'
    df.loc[valid_subject, 'meu_17_1_subject'] = True

    # 标记condition
    # 时间条件（任期内或离任后6个月）
    time_cond = (
        df['离任日期'].isna() |  # 在任情况
        (df['日期'] <= df['离任日期'] + pd.DateOffset(months=6))  # 离任后6个月
    )
    # 减持方式条件
    method_cond = df['减持方式'].isin(['竞价交易', '大宗交易', '协议转让'])
    # 排除特殊原因
    reason_cond = ~df['拟减持原因'].isin(['司法强制执行', '继承', '遗赠', '依法分割财产'])
    valid_condition = time_cond & method_cond & reason_cond
    df.loc[valid_condition, 'meu_17_1_condition'] = True

    # 标记constraint（独立检查）
    # 计算每日减持数量
    df.sort_values(['公司简称', '股东', '日期'], inplace=True)
    df['prev_持股数量'] = df.groupby(['公司简称', '股东'])['持股数量'].shift(1)
    df['当日减持数量'] = (df['prev_持股数量'] - df['持股数量']).clip(lower=0)

    # 获取上年末持股基数
    df['year'] = df['日期'].dt.year
    # 生成年末基准数据
    year_end = df.groupby(['公司简称', '股东', 'year']).apply(lambda x: x.loc[x['日期'].idxmax()])
    base_df = year_end[['公司简称', '股东', 'year', '持股数量']].reset_index()
    base_df['base_year'] = base_df['year'] + 1
    # 合并基准数据
    df = df.merge(
        base_df[['公司简称', '股东', 'base_year', '持股数量']],
        left_on=['公司简称', '股东', 'year'],
        right_on=['公司简称', '股东', 'base_year'],
        how='left',
        suffixes=('', '_base')
    )
    # 计算年累计减持量
    df['年累计减持'] = df.groupby(['公司简称', '股东', 'year'])['当日减持数量'].cumsum()
    # 验证约束条件
    valid_constraint = (df['年累计减持'] <= df['持股数量_base'] * 0.25) & df['持股数量_base'].notna()
    df.loc[valid_constraint, 'meu_17_1_constraint'] = True

    # 清理中间列
    df.drop(['prev_持股数量', '当日减持数量', 'year', '持股数量_base', 'base_year', '年累计减持'], 
            axis=1, errors='ignore', inplace=True)

    return df

In [ ]:
df = check_meu_17_1(df)
df

---

## MEU_17_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 所持股份不超过1000股 |
| constrain | 可一次全部转让且不受前款转让比例限制 |
| contextual_info | nan |
| note | nan |
| relation | exclude |
| target | MEU_17_1 |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1701 |
| completion_tokens | 2813 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_17_2(df):
    '''
    检查MEU_17_2合规性：
    subject: 上市公司董监高
    condition: 所持股份不超过1000股
    constraint: 可一次全部转让且不受前款转让比例限制
    '''
    df = df.copy()
    
    # 初始化标记列
    df['meu_17_2_subject'] = False
    df['meu_17_2_condition'] = False
    df['meu_17_2_constraint'] = None  # 初始化为None便于布尔值填充
    
    # 1. 验证责任主体：上市公司董监高
    # 根据股东身份字段直接匹配
    valid_subject = df['股东身份'] == '董监高'
    df.loc[valid_subject, 'meu_17_2_subject'] = True
    
    # 2. 验证触发条件：持股数量≤1000股
    # 直接比较数值型字段
    valid_condition = df['持股数量'] <= 1000
    df.loc[valid_condition, 'meu_17_2_condition'] = True
    
    # 3. 验证约束条件：一次性全部转让
    # 通过当日/累计减持比例判断是否清仓
    valid_constraint = (
        (df['当日减持比例'] == 1.0) |  # 当日清仓
        (df['累计减持比例'] == 1.0)    # 累计清仓
    )
    df.loc[valid_constraint, 'meu_17_2_constraint'] = True
    df.loc[~valid_constraint, 'meu_17_2_constraint'] = False
    
    return df

In [ ]:
df = check_meu_17_2(df)
df

---

# Law Article 23

## MEU_23_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | nan |
| constrain | 不得融券卖出本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 实际执行单元 |
| comments | nan |
| prompt_tokens | 1693 |
| completion_tokens | 6991 |


### 代码实现

In [ ]:
import pandas as pd

def check_meu_23_1(df):
    '''
    检查MEU_23_1合规性：
    "subject": "上市公司大股东 | 董监高",
    "condition": NaN,
    "constraint": "不得融券卖出本公司股份",
    "contextual_info": NaN
    '''
    df = df.copy()

    # 初始化标记列
    df['meu_23_1_subject'] = False
    df['meu_23_1_condition'] = False  # 初始化为False，后续统一处理
    df['meu_23_1_constraint'] = None  # 初始化为None，后续用布尔值填充

    # 1. 标记valid的subject（大股东或董监高）
    # 判断逻辑：股东身份属于董监高/控股股东/实际控制人，或持股比例≥5%
    subject_criteria = (
        df['股东身份'].isin(['董监高', '控股股东', '实际控制人']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[subject_criteria, 'meu_23_1_subject'] = True

    # 2. 标记valid的condition（由于MEU中condition为NaN，视为无条件触发，所有行标记为True）
    df['meu_23_1_condition'] = True  # 无条件约束，所有行条件满足

    # 3. 标记valid的constraint（减持方式非融券卖出）
    # 独立检查：无论subject是否满足，只要减持方式是融券卖出即违规
    constraint_met = (df['减持方式'] != '融券卖出')
    df['meu_23_1_constraint'] = constraint_met  # True表示合规，False表示违规

    return df

In [ ]:
df = check_meu_23_1(df)
df

---